In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

#Настройки показа графиков в блокноте
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 85)
pd.set_option('display.max_columns', 8)

from sklearn import preprocessing
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

#Стандартизация данных
from sklearn import preprocessing

#Построение диаграмм рассеивания
from pandas.plotting import scatter_matrix

#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

#Деревья решений для задачи классификации
from sklearn.tree import DecisionTreeClassifier

#Калибровка деревьев решений
from sklearn.calibration import CalibratedClassifierCV

df = pd.read_csv('train.csv')
trd = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')
td = pd.concat([trd, tsd], ignore_index=True, sort  = False)

In [2]:
df.shape

(891, 12)

In [3]:
df.drop(['Cabin', 'Name', 'Ticket','Embarked', 'SibSp', 'Parch', 'Fare', 'Age', 'Pclass'], axis=1, inplace=True)

In [4]:
df.head()

,PassengerId,Survived,Sex
0,1,0,male
1,2,1,female
2,3,1,female
3,4,1,female
4,5,0,male


In [5]:
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])

In [6]:
X_to_be_predicted = df[df.Survived.isnull()]
X_to_be_predicted = X_to_be_predicted.drop(['Survived'], axis = 1)

#Training data
train_data = df
train_data = train_data.dropna()
feature_train = train_data['Survived']
label_train = train_data.drop(['Survived'], axis = 1)
train_data.shape #891 x 28


(891, 3)

In [7]:
#Расщепление на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    # доля объёма тестового множества
                                                    test_size=0.2)

NameError: name 'X' is not defined

In [ ]:
model = RandomForestClassifier(random_state=42, #зерно датчика случайных чисел
                               n_estimators=30, #число деревьев в лесу
                               criterion='gini', #функция для дельта H, impurity ('gini' или 'entropy')
                               max_depth=5, #Макс число слоев                               
                               oob_score=True, #Вычислять out-of-bag ошибку                               
                               warm_start=False, #использовать результаты предыдущего вызова и нарастить предыдущий лес
                               class_weight=None #веса классов для балансировки выборки для обучения
                              )

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(metrics.classification_report(y_pred, y_test))
print('Out-of-bag score: {0}'.format(model.oob_score_)) 

pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
result = clf.predict(X_to_be_predicted)
submission = pd.DataFrame({'PassengerId':X_to_be_predicted.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)
filename = 'Titanic Predictions.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)